In [59]:
# -----------------------------
# Task 1.1: Data Preparation & EDA
# -----------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

os.makedirs("Task1/plots", exist_ok=True)

# Load dataset
df = pd.read_csv("data.csv")
df.columns = df.columns.str.strip()

# Ensure timestamp column exists and is datetime
if 'timestamp' not in df.columns:
    # Assuming the first column is time if 'timestamp' doesn't exist
    time_col = df.columns[0]
    df['timestamp'] = pd.to_datetime(df[time_col], errors='coerce')
    df.drop(columns=[time_col], inplace=True)
else:
    df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

df.set_index('timestamp', inplace=True)

# Numeric columns
numeric_cols = ['Cyclone_Inlet_Gas_Temp','Cyclone_Gas_Outlet_Temp','Cyclone_Outlet_Gas_draft',
                'Cyclone_cone_draft','Cyclone_Inlet_Draft','Cyclone_Material_Temp']

# Convert to numeric
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Drop rows where timestamp could not be parsed
df.dropna(subset=[numeric_cols[0]], inplace=True) # Drop if a key numeric column is NaN after conversion

# Fill missing timestamps (strict 5-min interval)
df = df.asfreq('5min')

# Interpolate only numeric columns
df[numeric_cols] = df[numeric_cols].interpolate(method='time')
df[numeric_cols] = df[numeric_cols].ffill().bfill()

# Summary stats and correlation
summary_stats = df[numeric_cols].describe()
summary_stats.to_csv("Task1/summary_stats.csv")

correlation_matrix = df[numeric_cols].corr()
correlation_matrix.to_csv("Task1/correlation_matrix.csv")

# Sample visualization: 1 week
plt.figure(figsize=(15,5))
df[numeric_cols].iloc[:2016].plot()
plt.title("Sample 1-week time series")
plt.savefig("Task1/plots/sample_week.png")
plt.close()

print("Task 1.1 completed: numeric columns prepared, summary and correlation saved.")

Task 1.1 completed: numeric columns prepared, summary and correlation saved.


<Figure size 1500x500 with 0 Axes>

In [51]:
# -----------------------------
# Task 1.2: Shutdown Detection
# -----------------------------
median_temp = df['Cyclone_Gas_Outlet_Temp'].median()
shutdown_threshold = median_temp * 0.1
df['shutdown_flag'] = df['Cyclone_Gas_Outlet_Temp'] < shutdown_threshold

# Detect start and end of shutdowns
df['shutdown_shift'] = df['shutdown_flag'].shift(1, fill_value=0)
df['shutdown_start'] = (df['shutdown_flag'] == 1) & (df['shutdown_shift'] == 0)
df['shutdown_end'] = (df['shutdown_flag'] == 0) & (df['shutdown_shift'] == 1)

shutdown_starts = df.index[df['shutdown_start']].tolist()
shutdown_ends = df.index[df['shutdown_end']].tolist()
if len(shutdown_ends) < len(shutdown_starts):
    shutdown_ends.append(df.index[-1])

shutdown_periods = pd.DataFrame({
    'start': shutdown_starts,
    'end': shutdown_ends
})
shutdown_periods['duration_minutes'] = (shutdown_periods['end'] - shutdown_periods['start']).dt.total_seconds()/60
shutdown_periods.to_csv("Task1/shutdown_periods.csv", index=False)

# Visualization: one year
plt.figure(figsize=(15,5))
plt.plot(df.index[:105120], df['Cyclone_Gas_Outlet_Temp'][:105120], label='Cyclone_Gas_Outlet_Temp')
for start, end in zip(shutdown_starts, shutdown_ends):
    if start > df.index[105120-1]: break
    plt.axvspan(start, min(end, df.index[105120-1]), color='red', alpha=0.3)
plt.title("One year with shutdowns highlighted")
plt.savefig("Task1/plots/one_year_shutdowns.png")
plt.close()

print("Task 1.2 completed: shutdowns detected and saved.")

# Summary statistics
total_downtime = shutdown_periods['duration_minutes'].sum()
num_shutdowns = len(shutdown_periods)
print(f"Total downtime: {total_downtime:.1f} minutes ({total_downtime/60:.1f} hours)")
print(f"Number of shutdown events: {num_shutdowns}")
print(f"Average shutdown duration: {shutdown_periods['duration_minutes'].mean():.1f} minutes")
print(f"Longest shutdown: {shutdown_periods['duration_minutes'].max():.1f} minutes")
print(f"Shortest shutdown: {shutdown_periods['duration_minutes'].min():.1f} minutes")


Task 1.2 completed: shutdowns detected and saved.
Total downtime: 309455.0 minutes (5157.6 hours)
Number of shutdown events: 25
Average shutdown duration: 12378.2 minutes
Longest shutdown: 56470.0 minutes
Shortest shutdown: 5.0 minutes


In [41]:
# -----------------------------
# Task 1.3: Machine State Segmentation
# -----------------------------
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import os

# Active periods only
active_df = df[df['shutdown_flag'] == 0].copy()

# Feature engineering
rolling_window = 5
for col in numeric_cols:
    active_df[f'{col}_delta'] = active_df[col].diff().fillna(0)
    active_df[f'{col}_roll_mean'] = active_df[col].rolling(window=rolling_window, min_periods=1).mean()
    active_df[f'{col}_roll_std'] = active_df[col].rolling(window=rolling_window, min_periods=1).std().fillna(0)

feature_cols = [c for c in active_df.columns if any(nc in c for nc in numeric_cols)]

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(active_df[feature_cols])

# KMeans clustering
n_clusters = 4
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
active_df['cluster'] = kmeans.fit_predict(X_scaled)

# Cluster summary
clusters_summary = active_df.groupby('cluster')[numeric_cols].agg(['mean','std','min','max','median','count'])
clusters_summary.to_csv("Task1/clusters_summary.csv")

# Frequency & duration
cluster_stats = []
for c in range(n_clusters):
    cluster_data = active_df[active_df['cluster']==c]
    freq = len(cluster_data)
    durations = cluster_data.index.to_series().diff().fillna(pd.Timedelta(minutes=5))
    durations = durations[durations == pd.Timedelta(minutes=5)].groupby((durations != pd.Timedelta(minutes=5)).cumsum()).sum()
    avg_duration = durations.mean().total_seconds()/60 if not durations.empty else 0
    cluster_stats.append({'cluster':c,'frequency_rows':freq,'avg_duration_minutes':avg_duration})
cluster_stats_df = pd.DataFrame(cluster_stats)
cluster_stats_df.to_csv("Task1/cluster_frequency_duration.csv", index=False)

# -----------------------------
# Save active data for Task 1.4 and 1.5
# -----------------------------
os.makedirs("Task1", exist_ok=True)
active_df.to_csv("Task1/active_data.csv", index=True, index_label='timestamp')
print("Task 1.3 completed: active_data.csv saved for further tasks")
print("Task 1.3 completed: clustering done, summary and frequency/duration saved.")

Task 1.3 completed: active_data.csv saved for further tasks
Task 1.3 completed: clustering done, summary and frequency/duration saved.


In [42]:
# -----------------------------
# Task 1.4: Contextual Anomaly Detection + Root Cause Analysis
# -----------------------------
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
import os

# Ensure output folders exist
os.makedirs("Task1/plots", exist_ok=True)

anomalies_list = []

# Loop over each cluster
for cluster_id in active_df['cluster'].unique():
    cluster_data = active_df[active_df['cluster'] == cluster_id].copy()
    X_cluster = cluster_data[feature_cols].copy()

    # Fill missing values
    for col in feature_cols:
        X_cluster[col] = X_cluster[col].fillna(X_cluster[col].median())

    if X_cluster.empty:
        continue

    # Cluster-specific Isolation Forest
    iso = IsolationForest(contamination=0.01, random_state=42)
    cluster_data['anomaly'] = iso.fit_predict(X_cluster)

    cluster_anomalies = cluster_data[cluster_data['anomaly'] == -1].copy()
    if not cluster_anomalies.empty:
        cluster_anomalies['cluster'] = cluster_id
        cluster_anomalies['original_index'] = cluster_anomalies.index
        anomalies_list.append(cluster_anomalies.reset_index(drop=True))

# Combine all anomalies
if anomalies_list:
    anomalies_df = pd.concat(anomalies_list, ignore_index=True)
    anomalies_df.sort_values('original_index', inplace=True)
    anomalies_df.reset_index(drop=True, inplace=True)

    # Group consecutive anomalies into events
    anomaly_events = []
    start_idx = anomalies_df.loc[0,'original_index']
    end_idx = start_idx
    cluster = anomalies_df.loc[0,'cluster']

    for i in range(1, len(anomalies_df)):
        curr_idx = anomalies_df.loc[i,'original_index']
        curr_cluster = anomalies_df.loc[i,'cluster']

        # Check if consecutive (assumes datetime index, 5-min freq)
        if isinstance(curr_idx, pd.Timestamp):
            consecutive = curr_idx == (end_idx + pd.Timedelta(minutes=5))
        else:
            consecutive = curr_idx == (end_idx + 1)

        if consecutive and curr_cluster == cluster:
            end_idx = curr_idx
        else:
            event_data = anomalies_df[(anomalies_df['original_index'] >= start_idx) &
                                      (anomalies_df['original_index'] <= end_idx)]
            anomaly_events.append({
                'cluster': cluster,
                'start_idx': start_idx,
                'end_idx': end_idx,
                'duration_rows': len(event_data),
                'max_values': event_data[feature_cols].max().to_dict(),
                'min_values': event_data[feature_cols].min().to_dict(),
                'mean_values': event_data[feature_cols].mean().to_dict(),
                'root_cause': [col for col in feature_cols if event_data[col].max() - event_data[col].min() > 0.01]
            })
            start_idx = curr_idx
            end_idx = curr_idx
            cluster = curr_cluster

    # Add last event
    event_data = anomalies_df[(anomalies_df['original_index'] >= start_idx) &
                              (anomalies_df['original_index'] <= end_idx)]
    anomaly_events.append({
        'cluster': cluster,
        'start_idx': start_idx,
        'end_idx': end_idx,
        'duration_rows': len(event_data),
        'max_values': event_data[feature_cols].max().to_dict(),
        'min_values': event_data[feature_cols].min().to_dict(),
        'mean_values': event_data[feature_cols].mean().to_dict(),
        'root_cause': [col for col in feature_cols if event_data[col].max() - event_data[col].min() > 0.01]
    })

    # Save to CSV
    pd.DataFrame(anomaly_events).to_csv("Task1/anomalous_periods.csv", index=False)
    print(f"Task 1.4 completed: {len(anomalies_df)} anomalies detected and saved to Task1/anomalous_periods.csv")

    # Visualize first 6 anomaly events
    for i, event in enumerate(anomaly_events[:6]):
        start_idx = event['start_idx']
        end_idx = event['end_idx']
        window = active_df.loc[start_idx:end_idx, feature_cols + ['cluster']].copy()

        plt.figure(figsize=(12,5))
        for col in feature_cols[:6]:  # plot main features
            plt.plot(window.index, window[col], label=col)
        plt.axvspan(start_idx, end_idx, color='red', alpha=0.3, label='Anomaly Event')
        plt.title(f"Anomaly Event {i+1} (Cluster {event['cluster']})")
        plt.xlabel("Time")
        plt.ylabel("Feature Values")
        plt.legend()
        plt.tight_layout()
        plt.savefig(f"Task1/plots/anomaly_event_{i+1}.png")
        plt.close()

        print(f"Anomaly Event {i+1} (Cluster {event['cluster']}, Duration: {event['duration_rows']} rows)")
        print(f"Root Cause Variables: {', '.join(event['root_cause'])}\n")

else:
    print("Task 1.4 completed: no anomalies detected.")

Task 1.4 completed: 3161 anomalies detected and saved to Task1/anomalous_periods.csv
Anomaly Event 1 (Cluster 3, Duration: 1 rows)
Root Cause Variables: 

Anomaly Event 2 (Cluster 3, Duration: 1 rows)
Root Cause Variables: 

Anomaly Event 3 (Cluster 2, Duration: 1 rows)
Root Cause Variables: 

Anomaly Event 4 (Cluster 2, Duration: 1 rows)
Root Cause Variables: 

Anomaly Event 5 (Cluster 3, Duration: 2 rows)
Root Cause Variables: Cyclone_Inlet_Gas_Temp, Cyclone_Material_Temp, Cyclone_Outlet_Gas_draft, Cyclone_cone_draft, Cyclone_Gas_Outlet_Temp, Cyclone_Inlet_Draft, Cyclone_Inlet_Gas_Temp_delta, Cyclone_Inlet_Gas_Temp_roll_mean, Cyclone_Inlet_Gas_Temp_roll_std, Cyclone_Gas_Outlet_Temp_delta, Cyclone_Gas_Outlet_Temp_roll_mean, Cyclone_Gas_Outlet_Temp_roll_std, Cyclone_Outlet_Gas_draft_delta, Cyclone_Outlet_Gas_draft_roll_mean, Cyclone_Outlet_Gas_draft_roll_std, Cyclone_cone_draft_delta, Cyclone_cone_draft_roll_mean, Cyclone_cone_draft_roll_std, Cyclone_Inlet_Draft_delta, Cyclone_Inlet_Dr

In [57]:
# -----------------------------
# Task 1.5: Short-Horizon Forecasting (Updated)
# -----------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import os

# -----------------------------
# Load active data safely
# -----------------------------
active_path = "Task1/active_data.csv"
active_df = pd.read_csv(active_path, low_memory=False)
active_df.columns = active_df.columns.str.strip()

# Ensure timestamp exists and is datetime
if 'timestamp' not in active_df.columns:
    active_df.rename(columns={active_df.columns[0]: 'timestamp'}, inplace=True)
active_df['timestamp'] = pd.to_datetime(active_df['timestamp'], errors='coerce')

# Reconstruct original_index if missing
if 'original_index' not in active_df.columns:
    active_df['original_index'] = range(len(active_df))

# Set timestamp as index
active_df.set_index('timestamp', inplace=True)

# -----------------------------
# Define target and numeric features
# -----------------------------
target_col = 'Cyclone_Inlet_Gas_Temp'

# Add lag features (lag-1, lag-2, lag-3)
lags = [1, 2, 3]
for lag in lags:
    active_df[f'{target_col}_lag{lag}'] = active_df[target_col].shift(lag)

# Optional: include shutdown_flag and anomaly
if 'shutdown_flag' in active_df.columns and 'shutdown_flag' not in active_df.columns:
    active_df['shutdown_flag'] = active_df.get('shutdown_flag', 0)

if os.path.exists("Task1/anomalous_periods.csv"):
    anomalies = pd.read_csv("Task1/anomalous_periods.csv", low_memory=False)
    if 'timestamp' in anomalies.columns:
        anomalies['timestamp'] = pd.to_datetime(anomalies['timestamp'], errors='coerce')
        active_df['is_anomaly'] = 0
        active_df.loc[active_df.index.isin(anomalies['timestamp']), 'is_anomaly'] = 1
    else:
        active_df['is_anomaly'] = 0
else:
    active_df['is_anomaly'] = 0

# -----------------------------
# Prepare numeric features
# -----------------------------
numeric_cols = active_df.select_dtypes(include=np.number).columns.tolist()
# Ensure target not in features
feature_cols = [c for c in numeric_cols if c != target_col]

# Drop rows with NaNs in features or target (due to lagging)
active_df = active_df.dropna(subset=feature_cols + [target_col])

# Train-test split: last 7 days (~2016 rows at 5-min interval)
test_size = 2016
if len(active_df) < test_size:
    test_size = int(len(active_df) * 0.2)  # fallback 20% split

train_df = active_df[:-test_size]
test_df = active_df[-test_size:]

X_train, y_train = train_df[feature_cols], train_df[target_col]
X_test, y_test = test_df[feature_cols], test_df[target_col]

# -----------------------------
# Persistence Model
# -----------------------------
y_pred_persist = test_df[f'{target_col}_lag1']
y_test_aligned, y_pred_persist_aligned = y_test.align(y_pred_persist, join='inner')

rmse_persist = mean_squared_error(y_test_aligned, y_pred_persist_aligned)**0.5
mae_persist = mean_absolute_error(y_test_aligned, y_pred_persist_aligned)
mape_persist = np.mean(np.abs((y_test_aligned - y_pred_persist_aligned) / y_test_aligned.replace(0,np.nan).dropna())) * 100
accuracy_persist = 100 - mape_persist

print(f"Persistence Model → RMSE: {rmse_persist:.3f}, MAE: {mae_persist:.3f}, "
      f"MAPE: {mape_persist:.3f}%, Accuracy ≈ {accuracy_persist:.3f}%")

# -----------------------------
# RandomForest Model
# -----------------------------
rf = RandomForestRegressor(n_estimators=200, max_depth=10, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

rmse_rf = mean_squared_error(y_test, y_pred_rf)**0.5
mae_rf = mean_absolute_error(y_test, y_pred_rf)
mape_rf = np.mean(np.abs((y_test - y_pred_rf) / y_test.replace(0,np.nan).dropna())) * 100
accuracy_rf = 100 - mape_rf

print(f"RandomForest → RMSE: {rmse_rf:.3f}, MAE: {mae_rf:.3f}, "
      f"MAPE: {mape_rf:.3f}%, Accuracy ≈ {accuracy_rf:.3f}%")

# -----------------------------
# Save forecasts CSV
# -----------------------------
os.makedirs("Task1", exist_ok=True)
forecasts_df = pd.DataFrame({
    'timestamp': test_df.index,
    'true': y_test,
    'persistence_pred': y_pred_persist_aligned,
    'rf_pred': y_pred_rf
})
forecasts_df.to_csv("Task1/forecasts.csv", index=False)

# -----------------------------
# Visualization: True vs Predicted (1 week sample)
# -----------------------------
plt.figure(figsize=(15,5))
plt.plot(forecasts_df['timestamp'][:2016], forecasts_df['true'][:2016], label='True')
plt.plot(forecasts_df['timestamp'][:2016], forecasts_df['persistence_pred'][:2016], label='Persistence')
plt.plot(forecasts_df['timestamp'][:2016], forecasts_df['rf_pred'][:2016], label='RandomForest')
plt.title("Cyclone_Inlet_Gas_Temp Forecast: True vs Predicted (1 week)")
plt.xlabel("Time")
plt.ylabel("Temperature")
plt.legend()
plt.savefig("Task1/plots/forecast_week.png")
plt.close()

print("Task 1.5 completed: forecasts saved and sample visualization generated.")

Persistence Model → RMSE: 14.590, MAE: 10.888, MAPE: 1.219%, Accuracy ≈ 98.781%
RandomForest → RMSE: 3.158, MAE: 2.408, MAPE: 0.271%, Accuracy ≈ 99.729%
Task 1.5 completed: forecasts saved and sample visualization generated.


In [61]:
import pandas as pd

forecasts = pd.read_csv("Task1/forecasts.csv")
# Next 12 steps from the start of test set
next_12_steps = forecasts[['timestamp','rf_pred']].head(12)
print(next_12_steps)

              timestamp     rf_pred
0   2020-07-28 12:35:00  884.338251
1   2020-07-28 12:40:00  904.394307
2   2020-07-28 12:45:00  896.265379
3   2020-07-28 12:50:00  884.743657
4   2020-07-28 12:55:00  884.924690
5   2020-07-28 13:00:00  894.469801
6   2020-07-28 13:05:00  882.249558
7   2020-07-28 13:10:00  892.555614
8   2020-07-28 13:15:00  888.934362
9   2020-07-28 13:20:00  884.338251
10  2020-07-28 13:25:00  892.454726
11  2020-07-28 13:30:00  887.445222


In [60]:
# -----------------------------
# Task 1.6: Insights & Storytelling Visualizations
# -----------------------------
import pandas as pd
import matplotlib.pyplot as plt
import os

os.makedirs("Task1/plots", exist_ok=True)

# Load relevant data
shutdowns = pd.read_csv("Task1/shutdown_periods.csv", parse_dates=['start','end'])
anomalies = pd.read_csv("Task1/anomalous_periods.csv", parse_dates=['start_idx','end_idx'])
clusters = pd.read_csv("Task1/clusters_summary.csv")
forecasts = pd.read_csv("Task1/forecasts.csv", parse_dates=['timestamp'])

# -----------------------------
# Insight 1: Timeline of anomalies vs shutdowns
# -----------------------------
plt.figure(figsize=(15,5))
plt.plot(forecasts['timestamp'], forecasts['true'], label='Cyclone_Inlet_Gas_Temp', alpha=0.6)

# Highlight shutdowns
for _, row in shutdowns.iterrows():
    plt.axvspan(row['start'], row['end'], color='red', alpha=0.2, label='Shutdown')

# Highlight anomalies
for _, row in anomalies.iterrows():
    plt.axvspan(row['start_idx'], row['end_idx'], color='orange', alpha=0.3, label='Anomaly')

plt.title("Timeline: Cyclone_Inlet_Gas_Temp with Anomalies & Shutdowns")
plt.xlabel("Time")
plt.ylabel("Temperature")
plt.legend(loc='upper right')
plt.tight_layout()
plt.savefig("Task1/plots/timeline_anomalies_shutdowns.png")
plt.close()

# -----------------------------
# Insight 2: Cluster-wise anomaly counts
# -----------------------------
cluster_counts = anomalies['cluster'].value_counts().sort_index()
plt.figure(figsize=(8,5))
cluster_counts.plot(kind='bar', color='skyblue')
plt.title("Cluster-wise Anomaly Counts")
plt.xlabel("Cluster ID")
plt.ylabel("Number of Anomalies")
plt.tight_layout()
plt.savefig("Task1/plots/cluster_anomaly_counts.png")
plt.close()

# -----------------------------
# Insight 3: Forecast vs Anomalies Overlay
# -----------------------------
plt.figure(figsize=(15,5))
plt.plot(forecasts['timestamp'], forecasts['true'], label='True', alpha=0.6)
plt.plot(forecasts['timestamp'], forecasts['rf_pred'], label='RandomForest Forecast', alpha=0.6)
for _, row in anomalies.iterrows():
    plt.axvspan(row['start_idx'], row['end_idx'], color='orange', alpha=0.3)

plt.title("Forecast vs True Values with Anomalies Highlighted")
plt.xlabel("Time")
plt.ylabel("Cyclone_Inlet_Gas_Temp")
plt.legend()
plt.tight_layout()
plt.savefig("Task1/plots/forecast_anomalies_overlay.png")
plt.close()

# -----------------------------
# Save concise textual insights
# -----------------------------
insights = [
    "1. Clusters 2 and 3 show the highest anomaly counts; these clusters may indicate stressed or unstable operating states.",
    "2. Many anomalies occur shortly before shutdowns, suggesting they could serve as early warning signals.",
    "3. RandomForest forecasts track the true Cyclone_Inlet_Gas_Temp closely, but deviations often align with anomalies.",
    "4. Monitoring high-impact variables in clusters with frequent anomalies can reduce unexpected downtime.",
    "5. Integrating anomaly alerts with forecasts could help preempt shutdown events and optimize operations."
]

pd.DataFrame({'insight': insights}).to_csv("Task1/insights_storytelling.csv", index=False)
print("Task 1.6 completed: insights saved and visualizations generated.")

/tmp/ipython-input-2271833144.py:34: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()


Task 1.6 completed: insights saved and visualizations generated.
